### .merge()

In [23]:
import pandas as pd

pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth",3000)
pd.set_option("display.max_rows", None)

In [3]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'name': ['Fred', 'Suzy', 'Suzy', 'Bob'],
                    'pet': ['Dog', 'Dog', 'Cat', 'Fish']})

df2 = pd.DataFrame({'Name': ['Suzy', 'Suzy', 'Suzy', 'Fred', 'Joe', 'Joe'],
                    'Color': ['Black', 'Blue', 'Red', 'Green', 'Yellow', 'Blue']})


In [4]:
display(df1), display(df2)

,name,pet
0,Fred,Dog
1,Suzy,Dog
2,Suzy,Cat
3,Bob,Fish


,Name,Color
0,Suzy,Black
1,Suzy,Blue
2,Suzy,Red
3,Fred,Green
4,Joe,Yellow
5,Joe,Blue


(None, None)

In [7]:
%%time
# be careful with names. Only because there was a white space after the column `name `, the code below didn't want to work
# (df1.merge(df2.assign(name=df2.Name) )) # by default it's an inner join

df1.merge(df2, left_on='name', right_on='Name')

CPU times: total: 0 ns
Wall time: 2 ms


,name,pet,Name,Color
0,Fred,Dog,Fred,Green
1,Suzy,Dog,Suzy,Black
2,Suzy,Dog,Suzy,Blue
3,Suzy,Dog,Suzy,Red
4,Suzy,Cat,Suzy,Black
5,Suzy,Cat,Suzy,Blue
6,Suzy,Cat,Suzy,Red


In [8]:
%%time
# merge and create new column `name` in df2 so we can merge on it with the df1 that has this column too. 
(df1.merge(df2.assign(name=df2.Name), how='left')) # all value from df1 will be brought

CPU times: total: 0 ns
Wall time: 3 ms


,name,pet,Name,Color
0,Fred,Dog,Fred,Green
1,Suzy,Dog,Suzy,Black
2,Suzy,Dog,Suzy,Blue
3,Suzy,Dog,Suzy,Red
4,Suzy,Cat,Suzy,Black
5,Suzy,Cat,Suzy,Blue
6,Suzy,Cat,Suzy,Red
7,Bob,Fish,NaN,NaN


In [9]:
%%time
df1 = (df1.merge(df2.assign(name=df2.Name), how='right')
    .assign(pet = df1.pet.fillna("no_pet"),
            name = df1.name.replace("Suzy", 'No_Suzy')))

CPU times: total: 15.6 ms
Wall time: 6 ms


In [10]:
%%time
df1 = (df1.merge(df2, left_on='name', right_on='Name', how='right')
    .assign(pet = df1.pet.fillna("no_pet"),
            name = df1.name.replace("Suzy", 'No_Suzy')))

CPU times: total: 0 ns
Wall time: 5 ms


In [26]:


# !pip install pyarrow fastparquet
df3 = (pd.read_parquet(r"data/9015-2013-11-12.parquet")
        .drop(columns=['accessionNumber', 'cikManager', 'managerName', 'periodOfReport',
        'submissionType', 'filedAsOfDate', 'xml_flag', 'updated_at', 'tr_type', 'tr_shares', 'tr_value',
        'quarter_as_int', 'prev_quarter_diff', 'next_quarter_diff',
        'quarter_max_filed', 'prev_shares_diff', 'prev_value_diff',
        'next_shares_diff', 'next_value_diff']))
df4 = pd.read_parquet(r"data/cusip_md.parquet")
df3.head()

,cusip,nameOfIssuer,titleOfClass,value,shares,shrsOrPrnAmt,putCall,quarter
0,Y8565J101,COM,COM,507000.0,15190.0,SH,None,2013Q3
1,Y8564W103,None,None,0.0,0.0,SH,None,2010Q2
2,Y8564W103,None,None,576000.0,25300.0,SH,None,2010Q1
3,Y8564W103,None,None,217000.0,9900.0,SH,None,2009Q3
4,Y8564W103,None,None,162000.0,11426.0,SH,None,2009Q1


In [25]:
df3.columns, df4.columns, df3.shape, df4.shape


(Index(['cusip', 'nameOfIssuer', 'titleOfClass', 'value', 'shares',
        'shrsOrPrnAmt', 'putCall', 'quarter'],
       dtype='object'),
 Index(['quarter', 'cusip', 'ticker', 'cusip_description', 'nameofissuer',
        'titleofclass'],
       dtype='object'),
 (76879, 8),
 (1914747, 6))

In [ ]:
# when merging, if left, the values that exist on the left but not on the right, will get displayed, but for the right df, there will be nan for the common column

df3.merge(df4, left_on=['quarter', 'cusip'], right_on=['quarter', 'cusip'], how='left')

In [ ]:
#different style of .merge()
(df3.merge(df4.assign(nameOfIssuer = df4.nameofissuer), how='left'))

In [ ]:

(df3.merge(df4, left_on=['quarter', 'cusip'], right_on=['quarter', 'cusip'], how='left' , indicator=True)) 